In [363]:
import numpy as np
import numpy as np
from scipy.linalg import eig
from numpy.linalg import cond

In [364]:
def print_results(eigenvalues, condition_number):
    print("Eigenvalues:")
    print(eigenvalues)
    print("="*80)
    print("Condition Number: {:.4e}".format(condition_number))

In [365]:
def condition(matrix):
    condition_number = np.linalg.cond(matrix)
    print("Condition Number: {:.4e}".format(condition_number))
    return 

In [366]:
#Single basis function - Neural Network
def single_neural_net(x,weights,biases,i, sigma): 

    return sigma(weights[i]*x+biases[i])

#Basis function multiplication helper for quadrature
def double_neural_net(x,weights1, biases1,weights2, biases2,i,j, sigma):

    return single_neural_net(x,weights1,biases1,i,sigma)*single_neural_net(x,weights2,biases2,j,sigma)

#Basis function multiplication helper for M1t
def double_neural_net_M1t(x,weights1, biases1,weights2, biases2,i,j, sigma,L,delta): 

    return single_neural_net(x,weights1,biases1,i,sigma)*single_neural_net(x,weights2,biases2,j,sigma)*(((L/2+delta)-x)/delta)**2

#Basis function multiplication helper for M2t
def double_neural_net_M2t(x,weights1, biases1,weights2, biases2,i,j, sigma,L,delta): 

    return single_neural_net(x,weights1,biases1,i,sigma)*single_neural_net(x,weights2,biases2,j,sigma)*((x-(L/2-delta))/delta)**2

#Basis function multiplication helper for off-diagonal blocks
def double_neural_net_off_diag(x,weights1, biases1,weights2, biases2,i,j, sigma,L,delta): 

    return single_neural_net(x,weights1,biases1,i,sigma)*single_neural_net(x,weights2,biases2,j,sigma)*(((L/2+delta)-x)*((x-((L/2)-delta))))/(delta**2)

In [367]:
# #Basis function multiplication helper for basis functions at the window function crossover
# def double_neural_net_tilde(x,weights1, biases1,weights2, biases2,i,j, sigma,L,delta,window): 

#     return single_neural_net(x,weights1,biases1,i,sigma)*single_neural_net(x,weights2,biases2,j,sigma)*window(L,delta,x)**2

# #Basis function multiplication helper for basis functions at the window function crossover
# def double_neural_net_off_diag(x,weights1, biases1,weights2, biases2,i,j, sigma,L,delta,window): 

#     return single_neural_net(x,weights1,biases1,i,sigma)*single_neural_net(x,weights2,biases2,j,sigma)*window(L,delta,x)


In [368]:
def window_1(L,delta,x):
    return (((L/2+delta)-x)/delta)

def window_2(L,delta,x):
    return ((x-((L/2)-delta))/delta)

def window_3(L,delta,x):
    return ((L/2+delta)-x)*((x-((L/2)-delta)))/(delta**2)

In [369]:
#Function to build the mass matrix - M
from scipy.integrate import quad
from scipy.integrate import quadrature

def mass_matrix(num_points,func,sigma, weights1, biases1,weights2, biases2, n,mins,maxs):
    M = np.zeros([n, n])
    collocation_points = np.linspace(mins,maxs,num_points)
    for i in range(n):
        for j in range(n):
            M[i, j] = quad(func, mins, maxs,points = collocation_points, args=(weights1, biases1,weights2, biases2,i,j,sigma),limit=1000)[0]

    return M


def mass_matrix_quadrature(num_points, func, sigma, weights1, biases1, weights2, biases2, n, mins, maxs,tol=1e-10, rtol=1e-10, maxiter=50):
    M = np.zeros([n, n])
    collocation_points = np.linspace(mins, maxs, num_points)
    
    for i in range(n):
        for j in range(n):
            # Define a wrapped function to include additional arguments
            def wrapped_func(x):
                return func(x, weights1, biases1, weights2, biases2, i, j, sigma)
            
            M[i, j] = quadrature(wrapped_func, mins, maxs, tol=tol, rtol=rtol,maxiter=maxiter)[0]

    return M

In [370]:
#Function to build the mass matrix - M_it, i=1,2
from scipy.integrate import quad

def mass_matrix_separate_window(num_points,func,sigma, weights1, biases1,weights2, biases2, n,mins,maxs,L,delta):
    M = np.zeros([n, n])
    collocation_points = np.linspace(mins,maxs,num_points)
    for i in range(n):
        for j in range(n):
            M[i, j] = quad(func, mins, maxs,points=collocation_points, args=(weights1, biases1,weights2, biases2,i,j,sigma,L,delta),limit=1000)[0]

    return M

def mass_matrix_separate_window_quadrature(num_points, func, sigma, weights1, biases1, weights2, biases2, n, mins, maxs, L, delta,tol=1e-10, rtol=1e-10, maxiter=50):
    M = np.zeros([n, n])
    collocation_points = np.linspace(mins, maxs, num_points)

    for i in range(n):
        for j in range(n):
            # Define a wrapped function to include additional arguments
            def wrapped_func(x):
                return func(x, weights1, biases1, weights2, biases2, i, j, sigma, L, delta)
            
            M[i, j] = quadrature(wrapped_func, mins, maxs, tol=tol, rtol=rtol,maxiter=maxiter)[0]

    return M

In [371]:
#Function to build the mass matrix - M_it, i=1,2
from scipy.integrate import quad

def mass_matrix_window(num_points,func,sigma, weights1, biases1,weights2, biases2, n,mins,maxs,L,delta,window):
    M = np.zeros([n, n])
    collocation_points = np.linspace(mins,maxs,num_points)
    for i in range(n):
        for j in range(n):
            M[i, j] = quad(func, mins, maxs,points=collocation_points, args=(weights1, biases1,weights2, biases2,i,j,sigma,L,delta,window),limit=1000)[0]

    return M

def mass_matrix_window_quadrature(num_points, func, sigma, weights1, biases1, weights2, biases2, n, mins, maxs, L, delta, window,tol=1e-10, rtol=1e-10, maxiter=50):
    M = np.zeros([n, n])
    collocation_points = np.linspace(mins, maxs, num_points)

    for i in range(n):
        for j in range(n):
            # Define a wrapped function to include additional arguments
            def wrapped_func(x):
                return func(x, weights1, biases1, weights2, biases2, i, j, sigma, L, delta, window)
            
            M[i, j] = quadrature(wrapped_func, mins, maxs, tol=tol, rtol=rtol,maxiter=maxiter)[0]

    return M

In [372]:
def evaluate_matrices(n, L, delta, sigma, w1, b1, w2, b2,num_points,tol, rtol, maxiter):
    M1 = mass_matrix_quadrature(num_points,double_neural_net,sigma, w1, b1,w1, b1, n, 0,(L/2)-delta,tol=tol, rtol=rtol, maxiter=maxiter)

    M1t = mass_matrix_separate_window_quadrature(num_points,double_neural_net_M1t,sigma, w1, b1,w1, b1, n, L/2-delta,L/2+delta,L,delta,tol, rtol, maxiter)

    M2 = mass_matrix_quadrature(num_points,double_neural_net,sigma, w2, b2,w2, b2, n, L/2+delta,L,tol, rtol, maxiter)

    M2t = mass_matrix_separate_window_quadrature(num_points,double_neural_net_M2t,sigma, w2, b2,w2, b2, n, L/2-delta,L/2+delta,L,delta,tol, rtol, maxiter)

    M12 = mass_matrix_separate_window_quadrature(num_points,double_neural_net_off_diag,sigma, w1, b1,w2, b2, n, L/2-delta,L/2+delta,L,delta,tol, rtol, maxiter) # What weights does this need

    M21 = M12.T

    M = np.block([[M1 + M1t, M12], [M12.T, M2 + M2t]])

    return {
        'M1': M1,
        'M1t': M1t,
        'M2': M2,
        'M2t': M2t,
        'M12': M12,
        'M21': M21,
    }


In [373]:
def generate_analytical_matrices_sin_func(n, L, delta, w1, b1, w2, b2):

    # Initialize matrices
    M1 = np.zeros((n, n))
    M1t = np.zeros((n, n))
    M2 = np.zeros((n, n))
    M2t = np.zeros((n, n))
    M12 = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            # Compute for M1 and M1t
            wij = w1[i] - w1[j]
            vij = w1[i] + w1[j]
            bij = b1[i] - b1[j]
            cij = b1[i] + b1[j]
            if wij != 0:
                M1[i, j] = -(np.sin(0.5 * vij * L - vij * delta + cij) * wij -
                             np.sin(0.5 * wij * L - wij * delta + bij) * vij +
                             np.sin(bij) * vij - np.sin(cij) * wij) / (vij * wij)
                
                M1t[i, j] = -1 / (delta**2 * wij**3 * vij**3) * (
                    2 * (2 * np.sin(0.5 * wij * L - wij * delta + bij) * delta**2 * vij**3 * wij**2 -
                         2 * np.sin(0.5 * vij * L - vij * delta + cij) * delta**2 * vij**2 * wij**3 -
                         2 * np.cos(0.5 * wij * L - wij * delta + bij) * delta * vij**3 * wij +
                         2 * np.cos(0.5 * vij * L - vij * delta + cij) * delta * vij * wij**3 +
                         vij**3 * np.sin(0.5 * wij * L + wij * delta + bij) -
                         np.sin(0.5 * vij * L + vij * delta + cij) * wij**3 -
                         vij**3 * np.sin(0.5 * wij * L - wij * delta + bij) +
                         np.sin(0.5 * vij * L - vij * delta + cij) * wij**3))
                
            else:
                M1[i, j] = (np.cos(bij) * w1[i] * L - 2 * np.cos(bij) * w1[i] * delta +
                            np.sin(cij) - np.sin(L * w1[i] - 2 * delta * w1[i] + cij)) / (2 * w1[i])
                M1t[i, j] = 1 / (12 * delta**2 * w1[i]**3) * (
                    32 * np.cos(bij) * delta**3 * w1[i]**3 +
                    24 * delta**2 * np.sin(L * w1[i] - 2 * delta * w1[i] + cij) * w1[i]**2 -
                    12 * np.cos(L * w1[i] - 2 * delta * w1[i] + cij) * delta * w1[i] +
                    3 * np.sin(L * w1[i] + 2 * delta * w1[i] + cij) -
                    3 * np.sin(L * w1[i] - 2 * delta * w1[i] + cij))

            # Compute for M2 and M2t
            wij = w2[i] - w2[j]
            vij = w2[i] + w2[j]
            bij = b2[i] - b2[j]
            cij = b2[i] + b2[j]
            if wij != 0:
                M2[i, j] = -((np.sin(vij*L + cij)*wij -
                            np.sin(wij*L  + bij)*vij + 
                            np.sin(0.5*wij*L+wij*delta+bij)*vij - 
                            np.sin(0.5*vij*L+vij*delta+cij)*wij)) / (vij*wij)

                M2t[i, j]=1 / (delta**2 * wij**3 * vij**3) * (
                    2 * (2 * np.sin(0.5 * wij * L + wij * delta + bij) * delta**2 * vij**3 * wij**2 -
                         2 * np.sin(0.5 * vij * L + vij * delta + cij) * delta**2 * vij**2 * wij**3 +
                         2 * np.cos(0.5 * wij * L + wij * delta + bij) * delta * vij**3 * wij -
                         2 * np.cos(0.5 * vij * L + vij * delta + cij) * delta * vij * wij**3 -
                         vij**3 * np.sin(0.5 * wij * L + wij * delta + bij) +
                         np.sin(0.5 * vij * L + vij * delta + cij) * wij**3 +
                         vij**3 * np.sin(0.5 * wij * L - wij * delta + bij) -
                         np.sin(0.5 * vij * L - vij * delta + cij) * wij**3))
            else:
                M2[i, j]=(np.cos(bij) * w2[i] * L - 2 * np.cos(bij) * w2[i] * delta -
                            np.sin(2*L*w2[i]+cij) + np.sin(L * w2[i] + 2*delta*w2[i] +cij)) / (2 * w2[i])

                M2t[i, j]=-1 / (12 * delta**2 * w2[i]**3) * (
                    -32 * np.cos(bij) * delta**3 * w2[i]**3 +
                    24 * delta**2 * np.sin(L * w2[i] + 2 * delta * w2[i] + cij) * w2[i]**2 +
                    12 * np.cos(L * w2[i] + 2 * delta * w2[i] + cij) * delta * w2[i] -
                    3 * np.sin(L * w2[i] + 2 * delta * w2[i] + cij) +
                    3 * np.sin(L * w2[i] - 2 * delta * w2[i] + cij))

            # Compute for M12
            wij=w1[i] - w2[j]
            vij=w1[i] + w2[j]
            bij=b1[i] - b2[j]
            cij=b1[i] + b2[j]
            if wij != 0:
                M12[i, j]= -1 / (delta**2 * wij**3 * vij**3) * (
                    2 * (np.cos(0.5 * wij * L + wij * delta + bij) * delta * vij**3 * wij -
                         np.cos(0.5 * vij * L + vij * delta + cij) * delta * vij * wij**3 +
                         np.cos(0.5 * wij * L - wij * delta + bij) * delta * vij**3 * wij -
                         np.cos(0.5 * vij * L - vij * delta + cij) * delta * vij * wij**3 -
                         vij**3 * np.sin(0.5 * wij * L + wij * delta + bij) +
                         np.sin(0.5 * vij * L + vij * delta + cij) * wij**3 +
                         vij**3 * np.sin(0.5 * wij * L - wij * delta + bij) -
                         np.sin(0.5 * vij * L - vij * delta + cij) * wij**3))
            else:
                M12[i, j]= 1 / (12 * delta**2 * w1[i]**3) * (
                    16 * np.cos(bij) * delta**3 * w1[i]**3 +
                    6 * np.cos(L * w1[i] + 2 * delta * w1[i] + cij) * delta * w1[i] +
                    6 * np.cos(L * w1[i] - 2 * delta * w1[i] + cij) * delta * w1[i] -
                    3 * np.sin(L * w1[i] + 2 * delta * w1[i] + cij) +
                    3 * np.sin(L * w1[i] - 2 * delta * w1[i] + cij))

    # Form the full matrix
    M_full=np.block([[M1 + M1t, M12], [M12.T, M2 + M2t]])

    # Compute eigenvalues
    eigenvalues=eig(M_full)[0]

    # Compute condition number
    condition_number=cond(M_full)

    # print_results(eigenvalues, condition_number)

    # Return results in a dictionary
    return {
        'M1': M1/2,
        'M1t': M1t/2,
        'M2': M2/2,
        'M2t': M2t/2,
        'M12': M12/2,
        'M21': M12.T/2,
    }


In [374]:
def compare_matrices(analytical, integrals):
    """
    Compares two lists of matrices to check if they are identical.

    Parameters:
    analytical (list): List of matrices from analytical solutions.
    integrals (list): List of matrices from integration solutions.

    Returns:
    list: A list of booleans indicating whether each pair of matrices is identical.
    """
    if len(analytical) != len(integrals):
        raise ValueError("Both lists must have the same number of matrices.")

    comparison_results = []
    
    for a_matrix, i_matrix in zip(analytical, integrals):
        if np.allclose(a_matrix, i_matrix):
            comparison_results.append(True)
        else:
            comparison_results.append(False)
    
    return comparison_results

In [375]:
def compare_arrays(array1, array2):
    """
    Compare two 2D arrays and print the indices and values of differing elements.
    
    Parameters:
    array1 (np.ndarray): The first array to compare.
    array2 (np.ndarray): The second array to compare.
    """
    # Check if the arrays have the same shape
    if array1.shape != array2.shape:
        raise ValueError("The arrays must have the same shape.")
    
    # Find the indices where the arrays are not close
    difference_indices = np.where(~np.isclose(array1, array2))

    # Iterate through the indices and print the differing values
    for i, j in zip(difference_indices[0], difference_indices[1]):
        print(f"Difference at index ({i}, {j}): array1 = {array1[i, j]}, array2 = {array2[i, j]}")


In [376]:
def compare_array_lists(list1, list2, rtol=1e-05, atol=1e-08):
    """
    Compare corresponding arrays in two lists of arrays and print the indices and values of differing elements.
    
    Parameters:
    list1 (list of np.ndarray): The first list of arrays to compare.
    list2 (list of np.ndarray): The second list of arrays to compare.
    rtol (float): The relative tolerance parameter for floating-point comparison (default is 1e-05).
    atol (float): The absolute tolerance parameter for floating-point comparison (default is 1e-08).
    """
    # Check if the lists have the same length
    if len(list1) != len(list2):
        raise ValueError("Both lists must have the same number of arrays.")

    # Iterate over pairs of arrays from both lists
    for idx, (array1, array2) in enumerate(zip(list1, list2)):
        # Check if the arrays have the same shape
        if array1.shape != array2.shape:
            raise ValueError(f"Arrays at index {idx} do not have the same shape.")
        
        # Find the indices where the arrays are not close
        difference_indices = np.where(~np.isclose(array1, array2, rtol=rtol, atol=atol))

        # Print the differing values for this pair of arrays
        print(f"Differences in array pair {idx}:")
        for i, j in zip(difference_indices[0], difference_indices[1]):
            print(f"  Difference at index ({i}, {j}): array1 = {array1[i, j]}, array2 = {array2[i, j]}")
        print()  # Blank line for readability between array pairs

In [377]:
def compare_named_array_lists(dict1, dict2, rtol=1e-05, atol=1e-08):
    """
    Compare corresponding arrays in two dictionaries of named arrays and print the indices and values of differing elements.
    
    Parameters:
    dict1 (dict of str: np.ndarray): The first dictionary of named arrays to compare.
    dict2 (dict of str: np.ndarray): The second dictionary of named arrays to compare.
    rtol (float): The relative tolerance parameter for floating-point comparison (default is 1e-05).
    atol (float): The absolute tolerance parameter for floating-point comparison (default is 1e-08).
    """
    # Check if the dictionaries have the same keys
    if set(dict1.keys()) != set(dict2.keys()):
        raise ValueError("Both dictionaries must have the same keys.")
    
    # Iterate over keys and corresponding arrays from both dictionaries
    for key in dict1.keys():
        array1 = dict1[key]
        array2 = dict2[key]
        
        # Check if the arrays have the same shape
        if array1.shape != array2.shape:
            raise ValueError(f"Arrays with key '{key}' do not have the same shape.")
        
        # Find the indices where the arrays are not close
        difference_indices = np.where(~np.isclose(array1, array2, rtol=rtol, atol=atol))

        # Print the differing values for this pair of arrays
        print(f"Differences in array with key '{key}':")
        for i, j in zip(difference_indices[0], difference_indices[1]):
            print(f"  Difference at index ({i}, {j}): array1 = {array1[i, j]}, array2 = {array2[i, j]}")
        print()  # Blank line for readability between array pairs

In [378]:
# Generate randomly weights and biases and split into two
np.random.seed(0)
n = 32 #half the number of neurons
R = 1 #sample range -R,R
num_points = 1000
b = np.random.uniform(-R, R, 2 * n)
w = np.random.uniform(-R, R, 2 * n)
b1, b2 = b[:n], b[n:]
w1, w2 = w[:n], w[n:]

L = np.pi  # size of the interval
delta = L / 8  # overlap

sigma = np.sin #activation function

tol= 1e-10
rtol = 1e-10
maxiter= 10000

In [379]:
analytical = generate_analytical_matrices_sin_func(n, L, delta, w1, b1, w2, b2)
integrals = evaluate_matrices(n, L, delta, sigma, w1, b1, w2, b2,num_points,tol,rtol,maxiter)

In [380]:
compare_named_array_lists(analytical, integrals)

Differences in array with key 'M1':

Differences in array with key 'M1t':

Differences in array with key 'M2':

Differences in array with key 'M2t':
  Difference at index (9, 28): array1 = -0.15721261830339323, array2 = -0.12195091533523227
  Difference at index (28, 9): array1 = -0.15721261830339323, array2 = -0.12195091533523227

Differences in array with key 'M12':
  Difference at index (19, 23): array1 = -0.035825858453543885, array2 = -0.03627313298064215
  Difference at index (24, 9): array1 = -0.02931938709777505, array2 = -0.029325756406506278
  Difference at index (24, 28): array1 = 0.2086032687347114, array2 = 0.20860064225148633

Differences in array with key 'M21':
  Difference at index (9, 24): array1 = -0.02931938709777505, array2 = -0.029325756406506278
  Difference at index (23, 19): array1 = -0.035825858453543885, array2 = -0.03627313298064215
  Difference at index (28, 24): array1 = 0.2086032687347114, array2 = 0.20860064225148633



In [381]:
Differences in array with key 'M1':

Differences in array with key 'M1t':

Differences in array with key 'M2':

Differences in array with key 'M2t':
  Difference at index (9, 28): array1 = -0.15721261830339323, array2 = -0.12195091533523199
  Difference at index (28, 9): array1 = -0.15721261830339323, array2 = -0.12195091533523199

Differences in array with key 'M12':
  Difference at index (19, 23): array1 = -0.035825858453543885, array2 = -0.03627313298064224
  Difference at index (24, 9): array1 = -0.02931938709777505, array2 = -0.029325756406505726
  Difference at index (24, 28): array1 = 0.2086032687347114, array2 = 0.20860064225148572

Differences in array with key 'M21':
  Difference at index (9, 24): array1 = -0.02931938709777505, array2 = -0.029325756406505726
  Difference at index (23, 19): array1 = -0.035825858453543885, array2 = -0.03627313298064224
  Difference at index (28, 24): array1 = 0.2086032687347114, array2 = 0.20860064225148572

SyntaxError: invalid syntax (3454859031.py, line 1)